In [7]:
from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
import dask.array as da
from glob import glob
import os
import numpy as np
import napari
from dask_image.imread import imread
from magicgui import magicgui
import sys
import matplotlib.pyplot as plt

#project_path = 'Z:\\PertzLab\\kvanunen\\data\\exp_kvu_134\\'
#project_path = 'Z:\\PertzLab\\rberthoz\\exp_035\\'
#project_path = 'Z:\\lhinder\\data\\rtm_mm_data\\exp_288\\'
#project_path = 'Z:\\PertzLab\\Ben\\exp_5\\'
project_path = 'Z:\\jfrei\\data\\experiments\\exp_51\\'


In [8]:
def tiff_to_lazy_da(path,folder,fov):
    file_name_pattern = str(fov).zfill(2)+"_*.tiff"
    filenames = sorted(glob(path + os.path.join(str(folder),file_name_pattern)), key=alphanumeric_key)
    print(file_name_pattern)
    # read the first file to get the shape and dtype
    # ASSUMES THAT ALL FILES SHARE THE SAME SHAPE/TYPE

    sample = imread(filenames[0])
    
    lazy_imread = delayed(imread)  # lazy reader
    lazy_arrays = [lazy_imread(fn) for fn in filenames]
    dask_arrays = [
        da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
        for delayed_reader in lazy_arrays
    ]
    # Stack into one large dask.array
    stack = da.stack(dask_arrays, axis=0)
    stack = np.squeeze(stack)
    return stack

In [9]:
nb_fovs = 48

#create the enum for the FOV selector widget
fov_choices = []
for fov in range(nb_fovs):
    fov_choices.append(("FOV "+str(fov), fov))

if not os.path.isdir(project_path):
    sys.exit("Cannot find project folder. Are you connected to the server?")

@magicgui(fov={"choices": fov_choices})
def load_fov(fov = 0):
    stack_raw = tiff_to_lazy_da(project_path, "raw", fov)
    stack_stim = tiff_to_lazy_da(project_path, "stim", fov)
    stack_mask = tiff_to_lazy_da(project_path, "mask", fov)
    #stack_mask_beads = tiff_to_lazy_da(project_path, 'labels_beads', fov)
    stack_light_mask = tiff_to_lazy_da(project_path, "light_mask", fov).astype('int')
    
    
    stack_raw_c0_layer.data = stack_raw[:,0,:,:]
    stack_raw_c1_layer.data = stack_raw[:,1,:,:]
    stack_stim_layer.data = stack_stim
    stack_mask_layer.data = stack_mask
    stack_light_mask_layer.data = stack_light_mask
    #stack_mask_beads_layer.data = stack_mask_beads.astype('int')
    
fov = 0
#init the viewer with FOV 0
stack_raw = tiff_to_lazy_da(project_path, "raw", fov)
stack_mask = tiff_to_lazy_da(project_path, "mask", fov)
stack_stim = np.squeeze(tiff_to_lazy_da(project_path, "stim", fov))
stack_light_mask = tiff_to_lazy_da(project_path, "light_mask", fov).astype('int')
#stack_mask_beads = tiff_to_lazy_da(project_path, 'labels_beads', fov)


viewer = napari.Viewer()
stack_raw_c0_layer = viewer.add_image(stack_raw[:,0,:,:], rgb=False,contrast_limits = (0,2000))
stack_raw_c1_layer = viewer.add_image(stack_raw[:,1,:,:], rgb=False,contrast_limits = (0,2000))
####layer = viewer.add_image(stack_raw, rgb=False,contrast_limits = (0,20000))
####stack_raw_layer = viewer.add_image(stack_raw, rgb=False,contrast_limits = (0,20000))
stack_stim_layer = viewer.add_image(stack_stim, rgb=False,contrast_limits = (0,2000))
stack_mask_layer = viewer.add_labels(stack_mask, opacity = 0.35)
stack_light_mask_layer = viewer.add_labels(stack_light_mask, opacity = 0.35)
#stack_mask_beads_layer = viewer.add_labels(stack_mask_beads, opacity = 0.35)




viewer.window.add_dock_widget(load_fov)





00_*.tiff
00_*.tiff
00_*.tiff
00_*.tiff


09_*.tiff
09_*.tiff
09_*.tiff
09_*.tiff
09_*.tiff
09_*.tiff
09_*.tiff
09_*.tiff
08_*.tiff
08_*.tiff
08_*.tiff
08_*.tiff
11_*.tiff
11_*.tiff
11_*.tiff
11_*.tiff
00_*.tiff
00_*.tiff
00_*.tiff
00_*.tiff
26_*.tiff
26_*.tiff
26_*.tiff
26_*.tiff
34_*.tiff
34_*.tiff
34_*.tiff
34_*.tiff
34_*.tiff
34_*.tiff
34_*.tiff
34_*.tiff


In [4]:
position_list = []

05_*.tiff
05_*.tiff
05_*.tiff
05_*.tiff


In [ ]:
from skimage.draw import polygon

viewer = napari.Viewer()
stack_raw_c0_layer = viewer.add_image(stack_raw[:,0,:,:], rgb=False,contrast_limits = (0,2000))
shapes_layer = viewer.add_shapes([], shape_type='polygon', edge_width=5,
                          edge_color='coral', face_color='royalblue')
def polygons_to_array(shapes_layer):
    '''Takes as input a napari shapes layer with polygons, and returns a binary array where polygon areas are 1 and background is 0'''
    img = np.zeros((1024, 1024), dtype=np.uint8)
    shapes = shapes_layer.data
    for shape in shapes:
        r = shape[:,0]
        c = shape[:,1]
        rr, cc = polygon(r, c)
        img[rr, cc] = 1
    return img

img = polygons_to_array(shapes_layer)
plt.imshow(img)



In [36]:
shapes_layer.

array([310.        , 229.        , 209.        , 221.        ,
       258.        , 300.        , 306.        , 268.        ,
       265.        , 298.        , 307.        , 307.        ,
       349.        , 399.67027588, 330.        , 330.        ])

In [ ]:
import mahotas.polygon

def run(n, reverse=0):
    canvas = np.zeros((n, n), dtype=int)
    lim = n-2
    print '\n%d x %d, lim=%d reverse=%d' % (n, n, lim, reverse)
    pts = [(1, 1), (1, lim), (lim, lim), (lim, 1), (1, 1)]
    if reverse:
        pts.reverse()
    mahotas.polygon.fill_polygon(pts, canvas)
    return canvas

for rev in (0, 1):
    for n in range(3, 14):
        print run(n, rev)

In [17]:
nb_fovs = 19

#create the enum for the FOV selector widget
fov_choices = []
for fov in range(nb_fovs):
    fov_choices.append(("FOV "+str(fov), fov))

if not os.path.isdir(project_path):
    sys.exit("Cannot find project folder. Are you connected to the server?")

@magicgui(fov={"choices": fov_choices})
def load_fov(fov = 0):
    stack_raw = tiff_to_lazy_da(project_path, "raw", fov)
    stack_raw_0_layer.data = stack_raw[:,0,:,:]
    stack_raw_1_layer.data = stack_raw[:,1,:,:]
    
    #stack_mask = tiff_to_lazy_da(project_path, "mask", fov)
    #stack_mask_layer.data = stack_mask

fov = 0
#init the viewer with FOV 0
stack_raw = tiff_to_lazy_da(project_path, "raw", fov)
#stack_mask = tiff_to_lazy_da(project_path, "mask", fov)


viewer = napari.Viewer()
stack_raw_0_layer = viewer.add_image(stack_raw[:,0,:,:], rgb=False,contrast_limits = (0,500))
stack_raw_1_layer = viewer.add_image(stack_raw[:,1,:,:], rgb=False,contrast_limits = (0,1000))
#stack_mask_layer = viewer.add_labels(stack_mask, opacity = 0.35_1)

viewer.window.add_dock_widget(load_fov)

00_*.tiff


In [124]:
import pandas as pd
# create a dataframe
df = pd.DataFrame({
    'Olympics': [2008, 2008, 2012, 2012, 2012, 2016, 2016, 2016],
    'Event': ['100 m', '200 m', '100 m', '200 m', '4x100 m', '200 m', '200 m', '4x100 m'],
    'Medal': ['Gold', 'Gold', 'Gold', 'Gold', 'Gold', 'Gold', 'Gold', 'Gold']
})
# display the datafram
print(df)
print(df['Event'].value_counts())

   Olympics    Event Medal
0      2008    100 m  Gold
1      2008    200 m  Gold
2      2012    100 m  Gold
3      2012    200 m  Gold
4      2012  4x100 m  Gold
5      2016    200 m  Gold
6      2016    200 m  Gold
7      2016  4x100 m  Gold
200 m      4
4x100 m    2
100 m      2
Name: Event, dtype: int64


'200 m'

In [121]:
for s in series:
    print(s)

TypeError: cannot unpack non-iterable int object

In [6]:
from skimage import io
folder = 'opto_check'
file_name_pattern = str(fov).zfill(2)+"_*.tiff"
filenames = sorted(glob(project_path + os.path.join(str(folder),file_name_pattern)), key=alphanumeric_key)

frame = io.imread(filenames[0])

channels_to_acq = ['c0','c1','c2']
plt.figure(figsize = (10,5),dpi = 300)
fig, axs = plt.subplots(2, frame.shape[2], figsize = (10,10),dpi = 300)
fig.suptitle('Acquisition settings preview')
for channel_nb, channel in enumerate(channels_to_acq):
    axs[0][channel_nb].imshow(frame[:,:,channel_nb],cmap = 'gray_r')
    axs[0][channel_nb] = channel
for channel_nb, channel in enumerate(channels_to_acq):
    flat = frame[:,:,channel_nb].flatten()
    axs[1][channel_nb].hist(flat, bins=150, range=(0, 65535/100))
plt.tight_layout()
plt.show()

IndexError: list index out of range

In [ ]:
frame.shape

In [ ]:
path = 'Z:\\lhinder\\data\\rtm_mm_data\\exp_26\\'
fov = 0
folder = 'stim'
folder = 'light_mask'
file_name_pattern = str(fov).zfill(2)+"_*.tiff"
filenames = sorted(glob(path + os.path.join(str(folder),file_name_pattern)), key=alphanumeric_key)

#mask = imread('Z:\\lhinder\\data\\rtm_mm_data\\exp_16\\raw\\100_01_0337.tiff')
#plt.imshow(mask[0,:,:])

In [ ]:
fov = 1
stack_raw = tiff_to_lazy_da(project_path, "raw", fov)
stack_mask = tiff_to_lazy_da(project_path, "mask", fov)
stack_light_mask = tiff_to_lazy_da(project_path, "light_mask", fov)
stack_stim = tiff_to_lazy_da(project_path, "stim", fov)

In [7]:
viewer = napari.Viewer()
viewer.add_image(stack_raw, rgb=False)
viewer.add_labels(stack_mask)
# change dimension order to CYX
#viewer.dims.order = (0,3,1,2)

<Labels layer 'stack_mask' at 0x24ccfb78a90>

In [8]:
viewer = napari.Viewer()
viewer.add_image(stack_stim, rgb=False)
viewer.add_labels(stack_light_mask)

<Labels layer 'stack_light_mask' at 0x24cdcfdacd0>

00_*.tiff
01_*.tiff
00_*.tiff


In [27]:
33%1

0

In [26]:
plt.imshow(stack_stim_apo[1,0,:,:])

NameError: name 'stack_stim_apo' is not defined

In [ ]:
from skimage.io import imsave
stack_raw = tiff_to_lazy_da(project_path, "raw", 11)
stack_loaded = stack_raw.compute()
imsave('Z:\\lhinder\\data\\rtm_mm_data\\exp_16\\fov_11.tiff',stack_loaded)

In [19]:
stack_light_mask

,Array,Chunk
Bytes,138.41 MB,4.19 MB
Shape,"(33, 1, 1024, 1024)","(1, 1, 1024, 1024)"
Count,132 Tasks,33 Chunks
Type,int32,numpy.ndarray
